In [ ]:
from secret_sauce import *
from lines_code import *
from my_widgets import *
from ipywidgets import fixed, interact
%matplotlib inline

# We are going to use two example images to test our code on! 
# Hold down the 'SHIFT' key and press 'ENTER' to see the images

show_images(white_lines, yellow_lines)

In [ ]:
# Let's find color thresholds! Hold down 'SHIFT' and press 'ENTER' again.
w = interact(show_color_threshold_results, red=red_widget, blue=blue_widget, green=green_widget)

In [ ]:
# Let's find edge thresholds! Hold down 'SHIFT' and press 'ENTER' again.
w = interact(show_edge_detection_results, low_threshold=low_widget, high_threshold=high_widget)

In [ ]:
#### Now we are going to put these thresholds together! 
#### Enter the numbers you got in the first two sections in the python code below

def threshold_image(image):
    
    low_red_threshold = 120
    high_red_threshold = 255

    low_blue_threshold = 120
    high_blue_threshold = 255

    low_green_threshold = 0
    high_green_threshold = 255

    low_edge_threshold = 20
    high_edge_threshold = 100

    #### You don't need to touch the rest of the code in this section, just fill in your numbers above!

    # The 'apply_masks' function will take in an image (for example, 'white_lines'), and output the
    # result of apply the color and edge masks!
    return apply_masks(image, 
                       low_red_threshold, high_red_threshold, 
                       low_blue_threshold, high_blue_threshold,
                       low_green_threshold, high_green_threshold, 
                       low_edge_threshold, high_edge_threshold)

show_images(white_lines, threshold_image(white_lines), yellow_lines, threshold_image(yellow_lines))

# To see the results, hold down 'SHIFT' and press 'ENTER' again.
# If the results don't look good, it's time to go back and change some of the numbers we've set so far

In [ ]:
# We only look at a small part of the image to find lane lines - no need to look for lane lines in the sky!
# We call this the "REGION OF INTEREST"
# Hold 'SHIFT' and press 'ENTER' to see the region in which we look for lane lines
show_regions_of_interest()

In [ ]:
# Now let's apply the region of interest to the masked images!
# The function 'region_of_interest' will block out the parts of the image we don't care about right now
# Hold 'SHIFT' and press 'ENTER' to see your results so far
white_masked_edges = region_of_interest(threshold_image(white_lines))
yellow_masked_edges = region_of_interest(threshold_image(yellow_lines))

show_images(white_lines, white_masked_edges, yellow_lines, yellow_masked_edges)

In [ ]:
white_masked_image = region_of_interest(threshold_image(white_lines))
yellow_masked_image = region_of_interest(threshold_image(yellow_lines))
w = interact(show_hough_lines, masked_image1=fixed(white_masked_edges), masked_image2=fixed(yellow_masked_edges),
            min_line_length=min_line_widget, max_line_gap=max_line_widget)

In [ ]:
def draw_lines(original_image, masked_image):
    # Now we combine lines from our HOUGH results above to get two lines corresponding to the lane lines
    # Insert the thresholds you found in the next few lines!
    
    # EDIT BELOW
    min_line_length = 40
    max_line_gap = 40
    
    color = (255, 0, 0) # Change the color of the final lines! 
        # (255, 0, 0) is red, (0, 255, 0) is blue, and (0, 0, 255) is green.
        # (255, 0, 255) is magenta, (0, 255, 255) is light blue
        # Try (80, 20, 255) for purple or (0, 255, 70) for teal
        # Or make up your own!
    
    thickness = 10
    # EDIT ABOVE

    # We get the hough lines using 'get_hough_lines'
    hough_lines = get_hough_lines(masked_image, min_line_length, max_line_gap)

    # Then we've written a function to find the two lane lines from these lines
    lines = get_lane_lines(hough_lines, masked_image.shape)
    
    line_image = np.copy(original_image) * 0
    for l in lines:
        cv2.line(line_image, (l[0][0], l[0][1]), (l[0][2], l[0][3]), color, thickness)
    return cv2.addWeighted(line_image, 2.0, original_image, 0.7, 0)

show_images(white_lines, draw_lines(white_lines, white_masked_edges),
           yellow_lines, draw_lines(yellow_lines, yellow_masked_edges))

In [ ]:
# Let's write one function that combines our other functions (by invoking them) to do this now!

def pipeline(image):
    
    # EDIT BELOW
    
    # Use our function to get the color and edge thresholded image
    # thresholded_image = ?
    thresholded_image = threshold_image(image)
    
    # Next, use our function to mask the thresholded_image based on the region of interest (ROI)
    # masked_image = ?
    masked_image = region_of_interest(thresholded_image)
    
    # Next, use our function to draw the lane lines on the image!
    # final_result = ?
    final_result = draw_lines(image, masked_image)
    
    return final_result

show_images(white_lines, pipeline(white_lines),
           yellow_lines, pipeline(yellow_lines))

In [ ]:
# Now let's run this on a video!
from moviepy.editor import VideoFileClip
from IPython.display import HTML
output = 'white.mp4'
clip = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip.fl_image(pipeline)
%time white_clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))